<a href="https://colab.research.google.com/github/jashvidesai/ORF-Thesis/blob/main/ALNSWithAllOperators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First portion is the initial soln, copied from the previous algorithm file.

In [1]:
import heapq
from collections import defaultdict
from heapq import heappush, heappop

In [2]:
import numpy as np
import heapq
import random
import itertools
from copy import deepcopy

# Problem Parameters
V = range(18)  # Nodes including depot
V_star = range(1, 18)  # Customer nodes
K = range(8)  # Vehicles
Q = [100, 120, 130, 105, 140, 95, 110, 115]  # Vehicle capacities
vehicle_speed = 60  # Speed in km/h
fixed_costs = [120, 138, 118, 122, 110, 110, 115, 113]  # Fixed costs per vehicle

np.random.seed(42)
distances = np.random.randint(10, 51, size=(len(V), len(V)))
for i in V:
    distances[i, i] = 0
    for j in range(i + 1, len(V)):
        distances[j, i] = distances[i, j]
for i in V:
    for j in V:
        for k in V:
            if distances[i, j] > distances[i, k] + distances[k, j]:
                distances[i, j] = distances[i, k] + distances[k, j]

t = (distances / vehicle_speed) * 60  # travel times in minutes
c = distances * 0.093  # travel costs (arbitrary scaling for fuel)

# Delivery and pickup demands
d = [0, 21, 11, 32, 34, 33, 10, 16, 20, 19, 28, 33, 35, 10, 37, 25, 36, 27]  # Delivery demands
p = [0, 23, 7, 22, 29, 35, 7, 28, 10, 26, 27, 6, 31, 6, 30, 21, 37, 13]  # Pickup demands

# Time windows and service times
a = [0, 15, 26, 20, 11, 14, 27, 7, 22, 29, 28, 17, 11, 29, 8, 17, 24, 5]  # Earliest arrival times
b = [300] * 18  # Latest departure times
s = [8, 8, 6, 5, 9, 9, 6, 4, 4, 9, 9, 8, 5, 3, 4, 9, 8, 7]  # Service times

# define a vehicle class to store critical information
class Vehicle:
    def __init__(self, vehicle_id, capacity, speed, cost, depot):
        self.vehicle_id = vehicle_id
        self.capacity = capacity
        self.speed = speed
        self.cost = cost
        self.route = [] # (customer_id, delivered, picked_up, arrival_time)
        self.current_time = 0

        # Updated Capacity Tracking
        self.full_vials = int(self.capacity * 0.75) # Start with 50% full vials
        self.empty_vials = 0 # Initially no empty vials
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials) # Available space
        self.current_location = depot

    def can_add_customer(self, node, travel_time):
        """
        Check if the vehicle can arrive within the time window.
        Returns (boolean) feasibility flag and expected arrival time.
        """
        # check if the vehicle has capacity
        if self.empty_vials == self.capacity:
            return False, self.current_time

        arrival_time = self.current_time + travel_time
        if arrival_time > b[node]:  # Too late
            return False, arrival_time
        if arrival_time < a[node]:  # Arrive early, wait
            arrival_time = a[node]
        return True, arrival_time

    def add_customer(self, node, delivery_demand, pickup_demand, travel_time):
        """
        Assigns a customer to the vehicle while ensuring delivery before pickup
        and allowing split deliveries.
        """
        # Step 1: Check feasibility (time window constraints)
        feasible, arrival_time = self.can_add_customer(node, travel_time)
        if not feasible:
            return False, delivery_demand, pickup_demand

        # Step 2: Deliver first, and update the vehicle load values
        delivered = min(self.full_vials, delivery_demand)
        self.full_vials -= delivered
        delivery_demand -= delivered
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 3: Pick up second, and update the vehicle load values
        picked_up = min(self.empty_space, pickup_demand)
        self.empty_vials += picked_up
        pickup_demand -= picked_up
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        # Step 4: Save customer in route & update time
        self.route.append((node, delivered, picked_up, arrival_time))
        self.current_time = arrival_time + s[node]
        self.current_location = node

        # ** Debugging Print Statements **
        print(f"Vehicle {self.vehicle_id} visited Customer {node}:")
        print(f"   - Delivered {delivered} (Remaining at customer: {delivery_demand})")
        print(f"   - Picked Up {picked_up} (Remaining at customer: {pickup_demand})")
        print(f"   - Vehicle State: Full Vials = {self.full_vials}, Empty Vials = {self.empty_vials}, Empty Space = {self.empty_space}")


        return True, delivery_demand, pickup_demand

    def return_to_depot(self, depot, t, depot_service_time=30):
        """
        Sends the vehicle back to the depot when full of empty vials.
        Resets load and allows for redeployment.
        """
        travel_time_to_depot = t[self.current_location][depot] # Retrieve from travel time matrix
        self.route.append((depot, 0, 0, self.current_time + travel_time_to_depot)) # Add depot return
        self.current_time += travel_time_to_depot + depot_service_time # Add travel and depot service time

        # Reset vehicle load: Start with 75% full vials, empty vials reset to 0
        self.full_vials = int(self.capacity * 0.75)
        self.empty_vials = 0
        self.empty_space = self.capacity - (self.full_vials + self.empty_vials)

        print(f"Vehicle {self.vehicle_id} returned to depot at time {self.current_time} and reset.")

# visiting order is based on distance to depot
def compute_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the farthest customer from the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = max(V_star, key=lambda c: distances[depot][c])  # Start with farthest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

# compute a new ordered route based on proximity to depot (after depot service)
def compute_nearest_ordered_route(V_star, depot, distances):
    """
    Computes a heuristic visiting order:
    1. Start with the closest customer to the depot.
    2. Sequentially add the nearest unvisited customer.
    """
    if not V_star:
        return []

    start_node = min(V_star, key=lambda c: distances[depot][c])  # Start with nearest
    ordered_route = [start_node]
    remaining_nodes = set(V_star) - {start_node}

    while remaining_nodes:
        last_node = ordered_route[-1]
        next_node = min(remaining_nodes, key=lambda c: distances[last_node][c])  # Nearest neighbor
        ordered_route.append(next_node)
        remaining_nodes.remove(next_node)

    return ordered_route

def generate_sequential_solution(V, V_star, K, Q, d, p, a, b, s, t, depot, distances):
    """
    Deploys vehicles sequentially, prioritizing larger vehicles first.
    If a vehicle is redeployed, it follows a new nearest-neighbor order.
    """
    initial_ordered_route = compute_ordered_route(V_star, depot, distances)  # Initial farthest-first order

    # Deploy largest vehicles first
    vehicles = sorted(
        [Vehicle(k, Q[k], vehicle_speed, fixed_costs[k], depot) for k in K],
        key=lambda v: v.capacity,
        reverse=True
    )

    remaining_deliveries = {i: d[i] for i in V_star} # dictionary to track remaining delivery demand
    remaining_pickups = {i: p[i] for i in V_star} # dictionary to track remaining pickup demand

    for vehicle in vehicles:
        print(f"\nDeploying Vehicle {vehicle.vehicle_id} (Capacity {vehicle.capacity})\n")

        current_route = initial_ordered_route[:]

        while True:  # Keep redeploying the vehicle until it can’t serve any more customers
            all_customers_served = True  # Flag to check if all customers are served

            # skips if fully served
            for customer in current_route[:]:
                if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
                    continue

                success, new_remaining_delivery, new_remaining_pickup = vehicle.add_customer(
                    customer, remaining_deliveries[customer], remaining_pickups[customer],
                    t[vehicle.current_location][customer]
                )

                if not success:
                    break

                remaining_deliveries[customer] = new_remaining_delivery
                remaining_pickups[customer] = new_remaining_pickup
                all_customers_served = False  # At least one customer was served

            vehicle.return_to_depot(depot, t, depot_service_time=30)  # 30-min depot service time

            # Check if there are still unfulfilled customers
            if all_customers_served:
                break  # No point in redeploying, so exit

            # Compute a new order for redeployment based on nearest-first heuristic
            unserved_customers = [i for i in V_star if remaining_deliveries[i] > 0 or remaining_pickups[i] > 0]
            current_route = compute_nearest_ordered_route(unserved_customers, depot, distances)

    return vehicles, remaining_deliveries, remaining_pickups

# Run the updated solution
vehicles_updated, remaining_deliveries, remaining_pickups = generate_sequential_solution(
    V, V_star, K, Q, d, p, a, b, s, t, depot=0, distances=distances
)

print("\n**Final Vehicle Routes and Loads**")
for vehicle in vehicles_updated:
    print(f"Vehicle {vehicle.vehicle_id}: Route {vehicle.route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

print("\n**Final Remaining Deliveries**")
for customer, remaining in remaining_deliveries.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to deliver")

print("\n**Final Remaining Pickups**")
for customer, remaining in remaining_pickups.items():
    if remaining > 0:
        print(f"Customer {customer}: {remaining} units left to pick up")


Deploying Vehicle 4 (Capacity 140)

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 6:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 7 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 62, Empty Vials = 42, Empty Space = 36
Vehicle 4 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 28, Empty Vials = 71, Empty Space = 41
Vehicle 4 visited Customer 11:
   - Delivered 28 (Remaining at customer: 5)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 77, Empty Space = 63
Vehicle 4 visited Customer 9:
   - Delivered 0 (Remaining at customer: 19)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 103, Empty Space = 37
Vehic

Removal Operators!

In [3]:
from copy import deepcopy
import random

def random_removal(solution, p, original_deliveries, original_pickups, t, s):
    modified_solution = deepcopy(solution)
    all_customers = set()

    # Step 1: Identify all customers
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:
                all_customers.add(stop[0])

    # Step 2: Randomly select customers to remove
    num_to_remove = int(p * len(all_customers))
    customers_to_remove = random.sample(list(all_customers), min(num_to_remove, len(all_customers)))

    # Step 3: Remove those customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Step 4: Initialize updated demand with original values
    updated_deliveries = {i: original_deliveries[i] for i in original_deliveries}
    updated_pickups = {i: original_pickups[i] for i in original_pickups}

    # Step 5: Re-run each remaining route from scratch, preserving order and skipping fulfilled customers
    for vehicle in modified_solution:
        saved_nodes = [stop[0] for stop in vehicle.route if stop[0] != 0]

        # Reset vehicle state
        vehicle.route = []
        vehicle.current_time = 0
        vehicle.full_vials = int(vehicle.capacity * 0.75)
        vehicle.empty_vials = 0
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)
        vehicle.current_location = 0

        vehicle.route.append((0, 0, 0, vehicle.current_time))

        for customer in saved_nodes:
            if updated_deliveries[customer] == 0 and updated_pickups[customer] == 0:
                continue

            success, new_d, new_p = vehicle.add_customer(
                customer,
                updated_deliveries[customer],
                updated_pickups[customer],
                t[vehicle.current_location][customer]
            )

            if success:
                updated_deliveries[customer] = new_d
                updated_pickups[customer] = new_p
                vehicle.current_time += s[customer]
            else:
                vehicle.return_to_depot(0, t)
                # reset and try again after depot
                success, new_d, new_p = vehicle.add_customer(
                    customer,
                    updated_deliveries[customer],
                    updated_pickups[customer],
                    t[vehicle.current_location][customer]
                )
                if success:
                    updated_deliveries[customer] = new_d
                    updated_pickups[customer] = new_p
                    vehicle.current_time += s[customer]

        # Final return to depot if not already there
        if vehicle.route[-1][0] != 0:
            vehicle.return_to_depot(0, t)

    # print statements
    print("Reprocessed Vehicle Routes After Random Removal (Vehicle ID, Time)")
    for vehicle in modified_solution:
        if len(vehicle.route) <= 1:
            continue
        formatted_route = [(int(n), round(float(t), 1)) for (n, d, p, t) in vehicle.route]
        print(f"Vehicle {vehicle.vehicle_id}: Route {formatted_route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

    print("**Removed Customers:", sorted(customers_to_remove))

    print("**Updated Remaining Deliveries:")
    for customer, val in updated_deliveries.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to deliver")

    print("**Updated Remaining Pickups:")
    for customer, val in updated_pickups.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to pick up")

    return modified_solution, customers_to_remove, updated_deliveries, updated_pickups

In [4]:
# Random Removal Example
original_d = deepcopy(d)
original_p = deepcopy(p)

original_d = {i: d[i] for i in range(len(d))}
original_p = {i: p[i] for i in range(len(p))}

modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    vehicles_updated, 0.3, original_d, original_p, t, s
)

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 6:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 7 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 62, Empty Vials = 42, Empty Space = 36
Vehicle 4 visited Customer 11:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 29, Empty Vials = 48, Empty Space = 63
Vehicle 4 visited Customer 9:
   - Delivered 19 (Remaining at customer: 0)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 10, Empty Vials = 74, Empty Space = 56
Vehicle 4 visited Customer 13:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 80, Empty Space = 60
Vehicle 4 visited Customer 7:
   - Deliver

In [5]:
# Related Removal
def related_removal(solution, p, original_deliveries, original_pickups, t, s, distances):
    modified_solution = deepcopy(solution)
    all_customers = set()

    # Step 1: Identify all customers in the solution (excluding depot)
    for vehicle in modified_solution:
        for stop in vehicle.route:
            if stop[0] != 0:
                all_customers.add(stop[0])

    if not all_customers:
        return modified_solution, [], original_deliveries, original_pickups

    # Step 2: Choose a seed and identify similar customers (close spatially)
    num_to_remove = int(p * len(all_customers))
    seed_customer = random.choice(list(all_customers))
    related_customers = sorted(all_customers, key=lambda c: distances[seed_customer][c])
    customers_to_remove = related_customers[:min(num_to_remove, len(related_customers))]

    # Step 3: Remove customers from routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Step 4: Reset demand to original
    updated_deliveries = {i: original_deliveries[i] for i in original_deliveries}
    updated_pickups = {i: original_pickups[i] for i in original_pickups}

    # Step 5: Reprocess routes (same manner as before)
    for vehicle in modified_solution:
        saved_nodes = [stop[0] for stop in vehicle.route if stop[0] != 0]

        vehicle.route = []
        vehicle.current_time = 0
        vehicle.full_vials = int(vehicle.capacity * 0.75)
        vehicle.empty_vials = 0
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)
        vehicle.current_location = 0

        vehicle.route.append((0, 0, 0, vehicle.current_time))

        for customer in saved_nodes:
            if updated_deliveries[customer] == 0 and updated_pickups[customer] == 0:
                continue

            success, new_d, new_p = vehicle.add_customer(
                customer,
                updated_deliveries[customer],
                updated_pickups[customer],
                t[vehicle.current_location][customer]
            )

            if success:
                updated_deliveries[customer] = new_d
                updated_pickups[customer] = new_p
                vehicle.current_time += s[customer]
            else:
                vehicle.return_to_depot(0, t)
                success, new_d, new_p = vehicle.add_customer(
                    customer,
                    updated_deliveries[customer],
                    updated_pickups[customer],
                    t[vehicle.current_location][customer]
                )
                if success:
                    updated_deliveries[customer] = new_d
                    updated_pickups[customer] = new_p
                    vehicle.current_time += s[customer]

        if vehicle.route[-1][0] != 0:
            vehicle.return_to_depot(0, t)

    # Print results
    print("Reprocessed Vehicle Routes After Related Removal (Vehicle ID, Time)")
    for vehicle in modified_solution:
        if len(vehicle.route) <= 1:
            continue
        formatted_route = [(int(n), round(float(t), 1)) for (n, d, p, t) in vehicle.route]
        print(f"Vehicle {vehicle.vehicle_id}: Route {formatted_route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

    print("**Removed Customers (Related):", sorted(customers_to_remove))

    print("**Updated Remaining Deliveries:")
    for customer, val in updated_deliveries.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to deliver")

    print("**Updated Remaining Pickups:")
    for customer, val in updated_pickups.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to pick up")

    return modified_solution, customers_to_remove, updated_deliveries, updated_pickups

In [6]:
# Related Removal Example
original_d = deepcopy(d)
original_p = deepcopy(p)

original_d = {i: d[i] for i in range(len(d))}
original_p = {i: p[i] for i in range(len(p))}

modified_related, removed_related, updated_deliveries, updated_pickups = related_removal(
    vehicles_updated, 0.3, original_d, original_p, t, s, distances
)

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 38, Empty Vials = 64, Empty Space = 38
Vehicle 4 visited Customer 11:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 5, Empty Vials = 70, Empty Space = 65
Vehicle 4 visited Customer 9:
   - Delivered 5 (Remaining at customer: 14)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 96, Empty Space = 44
Vehicle 4 visited Customer 8:
   - Delivered 0 (Remaining at customer: 20)
   - Picked Up 10 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 106, Empty Space = 34
Vehicle 4 visited Customer 13:
   - Delive

In [7]:
# Worst Removal
def worst_removal(solution, p, original_deliveries, original_pickups, t, s, distances, fixed_costs):
    modified_solution = deepcopy(solution)
    all_customers = set()
    customer_total_cost = {}

    # Extract all customers and calculate cost impact
    for vehicle in modified_solution:
        for i, stop in enumerate(vehicle.route):
            if stop[0] != 0:
                all_customers.add(stop[0])

                # Get previous and next stops
                prev_stop = vehicle.route[i - 1] if i > 0 else (0, 0, 0, 0)
                next_stop = vehicle.route[i + 1] if i < len(vehicle.route) - 1 else (0, 0, 0, 0) # Depot if last stop

                # Compute cost impact: added travel distance and vehicle fixed cost
                cost = (
                    distances[prev_stop[0]][stop[0]] +
                    distances[stop[0]][next_stop[0]] -
                    distances[prev_stop[0]][next_stop[0]] +
                    fixed_costs[vehicle.vehicle_id]
                )

                # Aggregate total cost contribution per customer
                customer_total_cost[stop[0]] = customer_total_cost.get(stop[0], 0) + cost

    if not all_customers:
        return modified_solution, [], remaining_deliveries, remaining_pickups

    num_to_remove = int(p * len(all_customers))

    # Sort customers by total cost contribution in descending order
    sorted_customers = sorted(customer_total_cost.items(), key=lambda x: x[1], reverse=True)

    # Extract the highest-cost customers for removal
    customers_to_remove = [customer for customer, _ in sorted_customers[:min(num_to_remove, len(sorted_customers))]]

    # Remove selected customers from all routes
    for vehicle in modified_solution:
        vehicle.route = [stop for stop in vehicle.route if stop[0] not in customers_to_remove]

    # Set remaining demand of removed customers to original
    updated_deliveries = {i: original_deliveries[i] for i in original_deliveries}
    updated_pickups = {i: original_pickups[i] for i in original_pickups}

    # Reprocess routes, just as before
    for vehicle in modified_solution:
        saved_nodes = [stop[0] for stop in vehicle.route if stop[0] != 0]

        vehicle.route = []
        vehicle.current_time = 0
        vehicle.full_vials = int(vehicle.capacity * 0.75)
        vehicle.empty_vials = 0
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)
        vehicle.current_location = 0

        vehicle.route.append((0, 0, 0, vehicle.current_time))

        for customer in saved_nodes:
            if updated_deliveries[customer] == 0 and updated_pickups[customer] == 0:
                continue

            success, new_d, new_p = vehicle.add_customer(
                customer,
                updated_deliveries[customer],
                updated_pickups[customer],
                t[vehicle.current_location][customer]
            )

            if success:
                updated_deliveries[customer] = new_d
                updated_pickups[customer] = new_p
                vehicle.current_time += s[customer]
            else:
                vehicle.return_to_depot(0, t)
                success, new_d, new_p = vehicle.add_customer(
                    customer,
                    updated_deliveries[customer],
                    updated_pickups[customer],
                    t[vehicle.current_location][customer]
                )
                if success:
                    updated_deliveries[customer] = new_d
                    updated_pickups[customer] = new_p
                    vehicle.current_time += s[customer]

        if vehicle.route[-1][0] != 0:
            vehicle.return_to_depot(0, t)

    # Print!
    print("Reprocessed Vehicle Routes After Worst Removal (Vehicle ID, Time)")
    for vehicle in modified_solution:
        if len(vehicle.route) <= 1:
            continue
        formatted_route = [(int(n), round(float(t), 1)) for (n, d, p, t) in vehicle.route]
        print(f"Vehicle {vehicle.vehicle_id}: Route {formatted_route}, Final Load: Full={vehicle.full_vials}, Empty={vehicle.empty_vials}")

    print("**Removed Customers (Worst):", sorted(customers_to_remove))

    print("**Updated Remaining Deliveries:")
    for customer, val in updated_deliveries.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to deliver")

    print("**Updated Remaining Pickups:")
    for customer, val in updated_pickups.items():
        if val > 0:
            print(f"Customer {customer}: {val} units left to pick up")

    return modified_solution, customers_to_remove, updated_deliveries, updated_pickups

In [8]:
# Worst Removal Example
original_d = deepcopy(d)
original_p = deepcopy(p)

original_d = {i: d[i] for i in range(len(d))}
original_p = {i: p[i] for i in range(len(p))}

modified_worst, removed_worst, updated_deliveries, updated_pickups = worst_removal(
    vehicles_updated, 0.3, original_d, original_p, t, s, distances, fixed_costs
)

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 6:
   - Delivered 10 (Remaining at customer: 0)
   - Picked Up 7 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 62, Empty Vials = 42, Empty Space = 36
Vehicle 4 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 28, Empty Vials = 71, Empty Space = 41
Vehicle 4 visited Customer 11:
   - Delivered 28 (Remaining at customer: 5)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 77, Empty Space = 63
Vehicle 4 visited Customer 9:
   - Delivered 0 (Remaining at customer: 19)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 103, Empty Space = 37
Vehicle 4 visited Customer 8:
   - Deliver

Insertion Operators!

In [12]:
'''
# Parallel Insertion
def parallel_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s):
    # Priority queue for selecting the best insertion based on lowest cost
    insertion_heap = []
    retry_customers = set()

    # Evaluate all removed customers across all vehicles and available routes
    for customer in removed_customers:
        for vehicle in solution:
            for idx in range(len(vehicle.route) - 1): # Iterate over current route positions

                # Extract current route stops
                prev_stop = vehicle.route[idx][0]  # Previous stop
                next_stop = vehicle.route[idx + 1][0]  # Next stop
                prev_arrival_time = vehicle.route[idx][3]  # Arrival time at previous stop

                # Compute travel time to the customer
                travel_time_to_customer = t[prev_stop][customer]
                arrival_time_customer = prev_arrival_time + travel_time_to_customer

                # Enforce time window constraints
                if arrival_time_customer > b[customer]: # Arrives too late
                    continue
                elif arrival_time_customer < a[customer]: # Arrives too early, must wait
                    arrival_time_customer = a[customer] # Adjust arrival time

                # Compute insertion cost
                insertion_cost = (c[prev_stop][customer] + c[customer][next_stop] - c[prev_stop][next_stop])

                # Compute max feasible delivery and pickup within vehicle capacity constraints
                max_delivery = min(remaining_deliveries.get(customer, 0), vehicle.full_vials)
                max_pickup = min(remaining_pickups.get(customer, 0), vehicle.empty_space)

                # If the vehicle can take any amount of the order, add to priority queue
                if max_delivery > 0 or max_pickup > 0:
                    heapq.heappush(insertion_heap, (insertion_cost, idx, vehicle.vehicle_id, vehicle, customer, max_delivery, max_pickup))

    # Process insertions in order of lowest cost
    while insertion_heap:
        _, idx, _, vehicle, customer, delivery, pickup = heapq.heappop(insertion_heap)

        # Skip if customer has already been fully served
        if remaining_deliveries.get(customer, 0) == 0 and remaining_pickups.get(customer, 0) == 0:
            continue

        # Assign feasible demand and update vehicle load
        assigned_delivery = min(remaining_deliveries[customer], vehicle.full_vials)
        assigned_pickup = min(remaining_pickups[customer], vehicle.empty_space)

        vehicle.full_vials -= assigned_delivery
        vehicle.empty_vials += assigned_pickup
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

        # Insert customer into route at the selected position
        vehicle.route.insert(idx + 1, (customer, assigned_delivery, assigned_pickup, vehicle.route[idx][3]))

        # Ensure sequential time values for future stops
        for i in range(idx + 2, len(vehicle.route)):
            prev_stop = vehicle.route[i - 1]
            next_stop = vehicle.route[i][0]
            travel_time = t[prev_stop[0]][next_stop]

            # Ensure no time jumps
            arrival_time = max(prev_stop[3] + travel_time, a[next_stop])
            vehicle.route[i] = (next_stop, vehicle.route[i][1], vehicle.route[i][2], arrival_time + s[next_stop])

        # Update remaining demand
        remaining_deliveries[customer] -= assigned_delivery
        remaining_pickups[customer] -= assigned_pickup

        # Remove customer from removed_customers if fully served
        if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
            removed_customers.remove(customer)

        # If customer demand is not fully met, add them back to the retry queue
        if remaining_deliveries[customer] > 0 or remaining_pickups[customer] > 0:
            retry_customers.add(customer)

    # Retry parallel insertion on unfulfilled customers using available vehicles
    if retry_customers:
        print(f"Retrying insertion for customers: {retry_customers}\n")
        return parallel_insertion(solution, retry_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s)

    # Try appending remaining customers to the end of existing routes
    unserved_customers = [
        cust for cust in remaining_deliveries
        if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0
    ]
    unserved_customers.sort(key=lambda c: t[0][c])

    for cust in unserved_customers[:]:
        inserted = False
        for vehicle in solution:
            if len(vehicle.route) <= 1:
                continue
            last_node = vehicle.route[-1][0]
            travel_time = t[last_node][cust]
            arrival_time = max(vehicle.current_time + travel_time, a[cust])

            if arrival_time > b[cust]:
                continue

            deliver = min(remaining_deliveries[cust], vehicle.full_vials)
            pickup = min(remaining_pickups[cust], vehicle.empty_space)

            if deliver > 0 or pickup > 0:
                vehicle.full_vials -= deliver
                vehicle.empty_vials += pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((cust, deliver, pickup, round(arrival_time, 2)))
                vehicle.current_time = round(arrival_time + s[cust], 2)

                remaining_deliveries[cust] -= deliver
                remaining_pickups[cust] -= pickup

                if remaining_deliveries[cust] == 0 and remaining_pickups[cust] == 0:
                    unserved_customers.remove(cust)
                inserted = True
                break

    # Handle Unused Vehicles Using Nearest Neighbor Heuristic
    unused_vehicles = [v for v in solution if len(v.route) == 1] # Only depot

    # Ensure both dicts have the same keys
    all_customers = set(remaining_deliveries.keys()) | set(remaining_pickups.keys())
    for cust in all_customers:
        remaining_deliveries.setdefault(cust, 0)
        remaining_pickups.setdefault(cust, 0)

    unserved_customers = [cust for cust in remaining_deliveries if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0]

    if unused_vehicles and unserved_customers:
        print("Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...")

        # Sort unused vehicles by largest capacity
        unused_vehicles.sort(key=lambda v: v.capacity, reverse=True)

        for vehicle in unused_vehicles:
            if not unserved_customers:
                break  # Exit if all demand is met

            vehicle.route = [(0, 0, 0, 0)]  # Start from depot
            vehicle.current_time = 0

            while True:
                feasible_customers = [
                    c for c in unserved_customers
                    if vehicle.current_time + t[vehicle.route[-1][0]][c] <= b[c]
                    and (remaining_deliveries[c] > 0 or remaining_pickups[c] > 0)
                ]

                if not feasible_customers:
                    break  # No more feasible customers for this vehicle

                next_customer = max(
                    feasible_customers,
                    key=lambda c: (-t[vehicle.route[-1][0]][c], remaining_deliveries[c] + remaining_pickups[c])
                )

                travel_time = t[vehicle.route[-1][0]][next_customer]
                arrival_time = max(vehicle.current_time + travel_time, a[next_customer])

                # Assign feasible delivery and pickup
                assigned_delivery = min(remaining_deliveries[next_customer], vehicle.full_vials)
                assigned_pickup = min(remaining_pickups[next_customer], vehicle.empty_space)

                # Skip if nothing to deliver/pick up
                if assigned_delivery == 0 and assigned_pickup == 0:
                    break  # Let another vehicle handle it later

                # Update vehicle load
                vehicle.full_vials -= assigned_delivery
                vehicle.empty_vials += assigned_pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((next_customer, assigned_delivery, assigned_pickup, round(arrival_time, 2)))
                vehicle.current_time = round(arrival_time + s[next_customer], 2)

                # Update demand
                remaining_deliveries[next_customer] -= assigned_delivery
                remaining_pickups[next_customer] -= assigned_pickup

                if remaining_deliveries[next_customer] == 0 and remaining_pickups[next_customer] == 0:
                    unserved_customers.remove(next_customer)

                # Stop if vehicle is full or out of time
                if vehicle.empty_space == 0 or vehicle.current_time > max(b):
                    break

            # Return to depot if any customers were served
            if len(vehicle.route) > 1:
                return_time = t[vehicle.route[-1][0]][0]
                vehicle.route.append((0, 0, 0, round(vehicle.current_time + return_time, 2)))

    for vehicle in solution:
        if len(vehicle.route) > 1:
            last = vehicle.route[-1][0]
            return_time = t[last][0]
            depot_arrival = vehicle.current_time + return_time
            vehicle.route.append((0, 0, 0, round(depot_arrival, 2)))

    return solution, remaining_deliveries, remaining_pickups

'''

'\n# Parallel Insertion\ndef parallel_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s):\n    # Priority queue for selecting the best insertion based on lowest cost\n    insertion_heap = []\n    retry_customers = set()\n\n    # Evaluate all removed customers across all vehicles and available routes\n    for customer in removed_customers:\n        for vehicle in solution:\n            for idx in range(len(vehicle.route) - 1): # Iterate over current route positions\n\n                # Extract current route stops\n                prev_stop = vehicle.route[idx][0]  # Previous stop\n                next_stop = vehicle.route[idx + 1][0]  # Next stop\n                prev_arrival_time = vehicle.route[idx][3]  # Arrival time at previous stop\n\n                # Compute travel time to the customer\n                travel_time_to_customer = t[prev_stop][customer]\n                arrival_time_customer = prev_arrival_time + travel

In [13]:
'''
# Initialize Delivery and Pickup
initial_deliveries = {i: d[i] for i in V_star}
initial_pickups = {i: p[i] for i in V_star}

# Run Random Removal First
removed_solution, removed_customers, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated),
    p=0.3,
    original_deliveries=deepcopy(initial_deliveries),
    original_pickups=deepcopy(initial_pickups),
    t=t,
    s=s
)

# Run Parallel Insertion to Reinsert Removed Customers
final_solution, final_deliveries, final_pickups = parallel_insertion(
    removed_solution,
    removed_customers,
    c=c,
    t=t,
    vehicle_capacities=Q,
    remaining_deliveries=updated_deliveries,
    remaining_pickups=updated_pickups,
    a=a,
    b=b,
    s=s
)

# Final Check
print("**FINAL DEMAND CHECK**")
unsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}
unsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}

if not unsatisfied_deliveries and not unsatisfied_pickups:
    print("✅ All demands have been fulfilled!")
else:
    print("⚠️ Remaining Demand:")
    if unsatisfied_deliveries:
        print("  - Deliveries:", unsatisfied_deliveries)
    if unsatisfied_pickups:
        print("  - Pickups:", unsatisfied_pickups)

# --- Print Final Routes ---
print("**Final Vehicle Routes After Re-Insertion**")
for vehicle in final_solution:
    if len(vehicle.route) > 1:
        print(f"Vehicle {vehicle.vehicle_id}: Route {[(c, d, p, float(t)) for (c, d, p, t) in vehicle.route]}")
'''

'\n# Initialize Delivery and Pickup\ninitial_deliveries = {i: d[i] for i in V_star}\ninitial_pickups = {i: p[i] for i in V_star}\n\n# Run Random Removal First\nremoved_solution, removed_customers, updated_deliveries, updated_pickups = random_removal(\n    deepcopy(vehicles_updated),\n    p=0.3,\n    original_deliveries=deepcopy(initial_deliveries),\n    original_pickups=deepcopy(initial_pickups),\n    t=t,\n    s=s\n)\n\n# Run Parallel Insertion to Reinsert Removed Customers\nfinal_solution, final_deliveries, final_pickups = parallel_insertion(\n    removed_solution,\n    removed_customers,\n    c=c,\n    t=t,\n    vehicle_capacities=Q,\n    remaining_deliveries=updated_deliveries,\n    remaining_pickups=updated_pickups,\n    a=a,\n    b=b,\n    s=s\n)\n\n# Final Check\nprint("**FINAL DEMAND CHECK**")\nunsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}\nunsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}\n\nif not u

In [14]:
'''
def solve_until_done(solution, removed_customers, c, t, Q, deliveries, pickups, a, b, s, max_iter=10):
    for iteration in range(max_iter):
        print(f"\n🔁 Parallel Insertion Iteration {iteration + 1}")
        prev_unmet_delivery = sum(deliveries[cust] for cust in deliveries if deliveries[cust] > 0)
        prev_unmet_pickup = sum(pickups[cust] for cust in pickups if pickups[cust] > 0)

        solution, deliveries, pickups = parallel_insertion(
            solution, removed_customers, c, t, Q, deliveries, pickups, a, b, s
        )

        new_unmet_delivery = sum(deliveries[cust] for cust in deliveries if deliveries[cust] > 0)
        new_unmet_pickup = sum(pickups[cust] for cust in pickups if pickups[cust] > 0)

        if new_unmet_delivery == 0 and new_unmet_pickup == 0:
            print("✅ All customer demands satisfied!")
            break

        if (new_unmet_delivery == prev_unmet_delivery and new_unmet_pickup == prev_unmet_pickup):
            print("⚠️ No improvement in unmet demand. Exiting early.")
            break

    return solution, deliveries, pickups
'''

'\ndef solve_until_done(solution, removed_customers, c, t, Q, deliveries, pickups, a, b, s, max_iter=10):\n    for iteration in range(max_iter):\n        print(f"\n🔁 Parallel Insertion Iteration {iteration + 1}")\n        prev_unmet_delivery = sum(deliveries[cust] for cust in deliveries if deliveries[cust] > 0)\n        prev_unmet_pickup = sum(pickups[cust] for cust in pickups if pickups[cust] > 0)\n\n        solution, deliveries, pickups = parallel_insertion(\n            solution, removed_customers, c, t, Q, deliveries, pickups, a, b, s\n        )\n\n        new_unmet_delivery = sum(deliveries[cust] for cust in deliveries if deliveries[cust] > 0)\n        new_unmet_pickup = sum(pickups[cust] for cust in pickups if pickups[cust] > 0)\n\n        if new_unmet_delivery == 0 and new_unmet_pickup == 0:\n            print("✅ All customer demands satisfied!")\n            break\n\n        if (new_unmet_delivery == prev_unmet_delivery and new_unmet_pickup == prev_unmet_pickup):\n            p

In [15]:
'''
# --- Example Usage ---
initial_deliveries = {i: d[i] for i in V_star}  # Original deliveries
initial_pickups = {i: p[i] for i in V_star}     # Original pickups

# 1. Run random removal
modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated), 0.3, deepcopy(initial_deliveries), deepcopy(initial_pickups), t, s
)

# 2. Run solve_until_done with parallel insertion
final_solution, final_deliveries, final_pickups = solve_until_done(
    modified_random, removed_random, c, t, Q,
    updated_deliveries, updated_pickups, a, b, s
)

# --- Final Check ---
print("\n**Demand Satisfaction Check**")
unsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}
unsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}

if not unsatisfied_deliveries and not unsatisfied_pickups:
    print("All customer demands are satisfied!")
else:
    print("Some customer demands remain unsatisfied!")
    if unsatisfied_deliveries:
        print("Remaining Delivery Demand:", unsatisfied_deliveries)
    if unsatisfied_pickups:
        print("Remaining Pickup Demand:", unsatisfied_pickups)

# --- Print Results ---
print("\n**Updated Routes After Parallel Insertion Loop**")
for vehicle in final_solution:
    print(f"Vehicle {vehicle.vehicle_id}: Route {[(c, d, p, float(t)) for (c, d, p, t) in vehicle.route]}")
'''

'\n# --- Example Usage ---\ninitial_deliveries = {i: d[i] for i in V_star}  # Original deliveries\ninitial_pickups = {i: p[i] for i in V_star}     # Original pickups\n\n# 1. Run random removal\nmodified_random, removed_random, updated_deliveries, updated_pickups = random_removal(\n    deepcopy(vehicles_updated), 0.3, deepcopy(initial_deliveries), deepcopy(initial_pickups), t, s\n)\n\n# 2. Run solve_until_done with parallel insertion\nfinal_solution, final_deliveries, final_pickups = solve_until_done(\n    modified_random, removed_random, c, t, Q,\n    updated_deliveries, updated_pickups, a, b, s\n)\n\n# --- Final Check ---\nprint("\n**Demand Satisfaction Check**")\nunsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}\nunsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}\n\nif not unsatisfied_deliveries and not unsatisfied_pickups:\n    print("All customer demands are satisfied!")\nelse:\n    print("Some customer demand

Parallel Insertion!

In [24]:
# Parallel Insertion
def core_parallel_insertion(solution, removed_customers, c, t, remaining_deliveries, remaining_pickups, a, b, s):
    insertion_heap = []
    retry_customers = set()

    for customer in removed_customers:
        for vehicle in solution:
            for idx in range(len(vehicle.route) - 1):
                prev_stop = vehicle.route[idx][0]
                next_stop = vehicle.route[idx + 1][0]
                prev_arrival_time = vehicle.route[idx][3]

                travel_time = t[prev_stop][customer]
                arrival_time = prev_arrival_time + travel_time

                if arrival_time > b[customer]:
                    continue
                elif arrival_time < a[customer]:
                    arrival_time = a[customer]

                cost = c[prev_stop][customer] + c[customer][next_stop] - c[prev_stop][next_stop]
                max_delivery = min(remaining_deliveries.get(customer, 0), vehicle.full_vials)
                max_pickup = min(remaining_pickups.get(customer, 0), vehicle.empty_space)

                if max_delivery > 0 or max_pickup > 0:
                    heapq.heappush(insertion_heap, (cost, idx, vehicle.vehicle_id, vehicle, customer, max_delivery, max_pickup))

    while insertion_heap:
        _, idx, _, vehicle, customer, delivery, pickup = heapq.heappop(insertion_heap)

        if remaining_deliveries.get(customer, 0) == 0 and remaining_pickups.get(customer, 0) == 0:
            continue

        assigned_delivery = min(remaining_deliveries[customer], vehicle.full_vials)
        assigned_pickup = min(remaining_pickups[customer], vehicle.empty_space)

        vehicle.full_vials -= assigned_delivery
        vehicle.empty_vials += assigned_pickup
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)


        prev_stop = vehicle.route[idx][0]
        prev_time = vehicle.route[idx][3]
        travel_time = t[prev_stop][customer]
        arrival_time = max(prev_time + travel_time, a[customer])
        service_end_time = float(round(arrival_time + s[customer], 2))

        vehicle.route.insert(idx + 1, (customer, assigned_delivery, assigned_pickup, float(round(service_end_time, 2))))

        for i in range(idx + 2, len(vehicle.route)):
            prev_stop = vehicle.route[i - 1]
            next_stop = vehicle.route[i][0]
            travel = t[prev_stop[0]][next_stop]
            arrival = max(prev_stop[3] + travel, a[next_stop])
            vehicle.route[i] = (next_stop, vehicle.route[i][1], vehicle.route[i][2], float(round(arrival + s[next_stop], 2)))

        remaining_deliveries[customer] -= assigned_delivery
        remaining_pickups[customer] -= assigned_pickup

        if remaining_deliveries[customer] > 0 or remaining_pickups[customer] > 0:
            retry_customers.add(customer)
        else:
            removed_customers.remove(customer)

    return solution, removed_customers, remaining_deliveries, remaining_pickups

def smart_parallel_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s, max_iter=10):
    attempt = 0
    unserved = removed_customers.copy()

    while attempt < max_iter and unserved:
        solution, unserved, remaining_deliveries, remaining_pickups = core_parallel_insertion(
            solution, unserved, c, t, remaining_deliveries, remaining_pickups, a, b, s
        )
        attempt += 1

    if unserved:
        print(f"\n🔁 Switching to fallback strategy after {attempt} core insertions")
        return full_parallel_insertion(solution, unserved, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s)

    for vehicle in solution:
        if len(vehicle.route) > 1:
            last = vehicle.route[-1][0]
            return_time = t[last][0]
            depot_arrival = float(vehicle.current_time + return_time)
            vehicle.route.append((0, 0, 0, float(round(depot_arrival, 2))))

    return solution, remaining_deliveries, remaining_pickups

In [25]:
# Full Parallel Insertion With Two Fallback Mechanisms
# Mechanism 1: Check to see if nodes with unserved demands can be added to the ends of existing routes
# Mechanism 2: If all else fails, do the NN Heuristic on the unserved nodes with the unutilized vehicles
# Parallel Insertion
def full_parallel_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s):
    # Priority queue for selecting the best insertion based on lowest cost
    insertion_heap = []
    retry_customers = set()

    # Evaluate all removed customers across all vehicles and available routes
    for customer in removed_customers:
        for vehicle in solution:
            for idx in range(len(vehicle.route) - 1): # Iterate over current route positions

                # Extract current route stops
                prev_stop = vehicle.route[idx][0]  # Previous stop
                next_stop = vehicle.route[idx + 1][0]  # Next stop
                prev_arrival_time = vehicle.route[idx][3]  # Arrival time at previous stop

                # Compute travel time to the customer
                travel_time_to_customer = t[prev_stop][customer]
                arrival_time_customer = prev_arrival_time + travel_time_to_customer

                # Enforce time window constraints
                if arrival_time_customer > b[customer]: # Arrives too late
                    continue
                elif arrival_time_customer < a[customer]: # Arrives too early, must wait
                    arrival_time_customer = a[customer] # Adjust arrival time

                # Compute insertion cost
                insertion_cost = (c[prev_stop][customer] + c[customer][next_stop] - c[prev_stop][next_stop])

                # Compute max feasible delivery and pickup within vehicle capacity constraints
                max_delivery = min(remaining_deliveries.get(customer, 0), vehicle.full_vials)
                max_pickup = min(remaining_pickups.get(customer, 0), vehicle.empty_space)

                # If the vehicle can take any amount of the order, add to priority queue
                if max_delivery > 0 or max_pickup > 0:
                    heapq.heappush(insertion_heap, (insertion_cost, idx, vehicle.vehicle_id, vehicle, customer, max_delivery, max_pickup))

    # Process insertions in order of lowest cost
    while insertion_heap:
        _, idx, _, vehicle, customer, delivery, pickup = heapq.heappop(insertion_heap)

        # Skip if customer has already been fully served
        if remaining_deliveries.get(customer, 0) == 0 and remaining_pickups.get(customer, 0) == 0:
            continue

        # Assign feasible demand and update vehicle load
        assigned_delivery = min(remaining_deliveries[customer], vehicle.full_vials)
        assigned_pickup = min(remaining_pickups[customer], vehicle.empty_space)

        vehicle.full_vials -= assigned_delivery
        vehicle.empty_vials += assigned_pickup
        vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

        # Insert customer into route at the selected position
        vehicle.route.insert(idx + 1, (customer, assigned_delivery, assigned_pickup, float(vehicle.route[idx][3])))

        # Ensure sequential time values for future stops
        for i in range(idx + 2, len(vehicle.route)):
            prev_stop = vehicle.route[i - 1]
            next_stop = vehicle.route[i][0]
            travel_time = t[prev_stop[0]][next_stop]

            # Ensure no time jumps
            arrival_time = max(prev_stop[3] + travel_time, a[next_stop])
            vehicle.route[i] = (next_stop, vehicle.route[i][1], vehicle.route[i][2], float(arrival_time + s[next_stop]))

        # Update remaining demand
        remaining_deliveries[customer] -= assigned_delivery
        remaining_pickups[customer] -= assigned_pickup

        # Remove customer from removed_customers if fully served
        if remaining_deliveries[customer] == 0 and remaining_pickups[customer] == 0:
            removed_customers.remove(customer)

        # If customer demand is not fully met, add them back to the retry queue
        if remaining_deliveries[customer] > 0 or remaining_pickups[customer] > 0:
            retry_customers.add(customer)

    # Try appending remaining customers to the end of existing routes
    unserved_customers = [
        cust for cust in remaining_deliveries
        if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0
    ]
    unserved_customers.sort(key=lambda c: t[0][c])

    for cust in unserved_customers[:]:
        inserted = False
        for vehicle in solution:
            if len(vehicle.route) <= 1:
                continue
            last_node = vehicle.route[-1][0]
            travel_time = t[last_node][cust]
            arrival_time = max(vehicle.current_time + travel_time, a[cust])

            if arrival_time > b[cust]:
                continue

            deliver = min(remaining_deliveries[cust], vehicle.full_vials)
            pickup = min(remaining_pickups[cust], vehicle.empty_space)

            if deliver > 0 or pickup > 0:
                vehicle.full_vials -= deliver
                vehicle.empty_vials += pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((cust, deliver, pickup, float(round(arrival_time, 2))))
                vehicle.current_time = round(arrival_time + s[cust], 2)

                remaining_deliveries[cust] -= deliver
                remaining_pickups[cust] -= pickup

                if remaining_deliveries[cust] == 0 and remaining_pickups[cust] == 0:
                    unserved_customers.remove(cust)
                inserted = True
                break

    # Handle Unused Vehicles Using Nearest Neighbor Heuristic
    unused_vehicles = [v for v in solution if len(v.route) == 1] # Only depot

    # Ensure both dicts have the same keys
    all_customers = set(remaining_deliveries.keys()) | set(remaining_pickups.keys())
    for cust in all_customers:
        remaining_deliveries.setdefault(cust, 0)
        remaining_pickups.setdefault(cust, 0)

    unserved_customers = [cust for cust in remaining_deliveries if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0]

    if unused_vehicles and unserved_customers:
        print("Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...")

        # Sort unused vehicles by largest capacity
        unused_vehicles.sort(key=lambda v: v.capacity, reverse=True)

        for vehicle in unused_vehicles:
            if not unserved_customers:
                break  # Exit if all demand is met

            vehicle.route = [(0, 0, 0, 0)]  # Start from depot
            vehicle.current_time = 0

            while True:
                feasible_customers = [
                    c for c in unserved_customers
                    if vehicle.current_time + t[vehicle.route[-1][0]][c] <= b[c]
                    and (remaining_deliveries[c] > 0 or remaining_pickups[c] > 0)
                ]

                if not feasible_customers:
                    break  # No more feasible customers for this vehicle

                next_customer = max(
                    feasible_customers,
                    key=lambda c: (-t[vehicle.route[-1][0]][c], remaining_deliveries[c] + remaining_pickups[c])
                )

                travel_time = t[vehicle.route[-1][0]][next_customer]
                arrival_time = max(vehicle.current_time + travel_time, a[next_customer])
                service_end_time = float(round(arrival_time + s[next_customer], 2))

                # Assign feasible delivery and pickup
                assigned_delivery = min(remaining_deliveries[next_customer], vehicle.full_vials)
                assigned_pickup = min(remaining_pickups[next_customer], vehicle.empty_space)

                # Skip if nothing to deliver/pick up
                if assigned_delivery == 0 and assigned_pickup == 0:
                    break  # Let another vehicle handle it later

                # Update vehicle load
                vehicle.full_vials -= assigned_delivery
                vehicle.empty_vials += assigned_pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((next_customer, assigned_delivery, assigned_pickup, float(round(service_end_time, 2))))
                vehicle.current_time = round(arrival_time + s[customer], 2)

                # Update demand
                remaining_deliveries[next_customer] -= assigned_delivery
                remaining_pickups[next_customer] -= assigned_pickup

                if remaining_deliveries[next_customer] == 0 and remaining_pickups[next_customer] == 0:
                    unserved_customers.remove(next_customer)

                # Stop if vehicle is full or out of time
                if vehicle.empty_space == 0 or vehicle.current_time > max(b):
                    break

            # Return to depot if any customers were served
            if len(vehicle.route) > 1:
                return_time = t[vehicle.route[-1][0]][0]
                vehicle.route.append((0, 0, 0, float(round(vehicle.current_time + return_time, 2))))

    for vehicle in solution:
        if len(vehicle.route) > 1:
            last = vehicle.route[-1][0]
            return_time = t[last][0]
            depot_arrival = vehicle.current_time + return_time
            vehicle.route.append((0, 0, 0, float(round(depot_arrival, 2))))

    return solution, remaining_deliveries, remaining_pickups

In [26]:
# Convert delivery and pickup lists to dictionaries for each customer
initial_deliveries = {i: d[i] for i in V_star}
initial_pickups = {i: p[i] for i in V_star}

# 1. Do random removal
modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated), 0.3, deepcopy(initial_deliveries), deepcopy(initial_pickups), t, s
)

# 2. Then try reinsertion using smart_parallel_insertion
final_solution, final_deliveries, final_pickups = smart_parallel_insertion(
    modified_random, removed_random, c, t, Q,
    updated_deliveries, updated_pickups, a, b, s
)

#3. Finally try the full parallel insertion
final_solution, final_deliveries, final_pickups = full_parallel_insertion(
    final_solution, removed_random, c, t, Q,
    updated_deliveries, updated_pickups, a, b, s
)

# --- Final Check ---
print("\n✅ Demand Satisfaction Check")
unsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}
unsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}

if not unsatisfied_deliveries and not unsatisfied_pickups:
    print("All customer demands are satisfied!")
else:
    print("⚠️ Some customer demands remain unsatisfied!")
    if unsatisfied_deliveries:
        print("Remaining Deliveries:", unsatisfied_deliveries)
    if unsatisfied_pickups:
        print("Remaining Pickups:", unsatisfied_pickups)

# --- Print Final Routes ---
print("\n📦 Updated Routes After Parallel Insertion Loop")
for vehicle in final_solution:
    if len(vehicle.route) > 1:
        route_str = [(n, d, p, round(arrival, 1)) for (n, d, p, arrival) in vehicle.route]
        print(f"Vehicle {vehicle.vehicle_id}: Route {route_str}")

Vehicle 4 visited Customer 5:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 35 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 72, Empty Vials = 35, Empty Space = 33
Vehicle 4 visited Customer 4:
   - Delivered 34 (Remaining at customer: 0)
   - Picked Up 29 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 38, Empty Vials = 64, Empty Space = 38
Vehicle 4 visited Customer 11:
   - Delivered 33 (Remaining at customer: 0)
   - Picked Up 6 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 5, Empty Vials = 70, Empty Space = 65
Vehicle 4 visited Customer 9:
   - Delivered 5 (Remaining at customer: 14)
   - Picked Up 26 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 96, Empty Space = 44
Vehicle 4 visited Customer 7:
   - Delivered 0 (Remaining at customer: 16)
   - Picked Up 28 (Remaining at customer: 0)
   - Vehicle State: Full Vials = 0, Empty Vials = 124, Empty Space = 16
Vehicle 4 visited Customer 16:
   - Delive

Regret Split Insertion!

In [11]:
# Regret Split Insertion
import heapq
from copy import deepcopy

def regret_split_insertion(solution, removed_customers, c, t, vehicle_capacities, remaining_deliveries, remaining_pickups, a, b, s):
    # Initialize variables
    uninsertable_customers = set()
    insertion_candidates = {}

    # Build a dictionary of insertion options for each customer
    for customer in removed_customers:
        positions = []

        for vehicle in solution:
            for idx in range(len(vehicle.route) - 1):
                prev_stop = vehicle.route[idx][0]
                next_stop = vehicle.route[idx + 1][0]
                prev_arrival_time = vehicle.route[idx][3]

                travel_time_to_customer = t[prev_stop][customer]
                arrival_time = prev_arrival_time + travel_time_to_customer

                # Enforce time window
                if arrival_time > b[customer]:
                    continue
                if arrival_time < a[customer]:
                    arrival_time = a[customer]

                # Capacity constraints
                feasible_delivery = min(remaining_deliveries[customer], vehicle.full_vials)
                feasible_pickup = min(remaining_pickups[customer], vehicle.empty_space)

                if feasible_delivery == 0 and feasible_pickup == 0:
                    continue

                # Compute insertion cost and score (cost per unit served)
                insertion_cost = (
                    c[prev_stop][customer] +
                    c[customer][next_stop] -
                    c[prev_stop][next_stop]
                )
                served_units = feasible_delivery + feasible_pickup
                score = insertion_cost / served_units

                positions.append((score, insertion_cost, idx, vehicle.vehicle_id, vehicle, feasible_delivery, feasible_pickup))

        if positions:
            # Sort positions by score (ascending)
            positions.sort()
            insertion_candidates[customer] = positions
        else:
            uninsertable_customers.add(customer) # there were no positions, so add it here. use later.

    # Compute regret values
    regret_heap = []
    for customer, options in insertion_candidates.items():
        if len(options) > 1:
            regret_value = options[1][1] - options[0][1]
        else:
            regret_value = float('inf') # Only one option, so max regret

        heapq.heappush(regret_heap, (-regret_value, customer)) # Max-heap (prioritize highest-regret)

    # Insert customers by regret order
    while regret_heap:
        _, customer = heapq.heappop(regret_heap)
        options = insertion_candidates[customer]
        total_delivery = remaining_deliveries[customer]
        total_pickup = remaining_pickups[customer]

        for score, cost, idx, _, vehicle, feasible_delivery, feasible_pickup in options:
            if total_delivery == 0 and total_pickup == 0:
                break

            # Recompute feasibility in case vehicle loads changed
            feasible_delivery = min(total_delivery, vehicle.full_vials)
            feasible_pickup = min(total_pickup, vehicle.empty_space)

            if feasible_delivery == 0 and feasible_pickup == 0:
                continue

            # Insert into route
            vehicle.full_vials -= feasible_delivery
            vehicle.empty_vials += feasible_pickup
            vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

            vehicle.route.insert(idx + 1, (customer, feasible_delivery, feasible_pickup, vehicle.current_time))

            # Update timings of future stops
            for i in range(idx + 2, len(vehicle.route)):
                prev_stop = vehicle.route[i - 1]
                next_stop = vehicle.route[i][0]
                travel_time = t[prev_stop[0]][next_stop]
                arrival_time = max(prev_stop[3] + travel_time, a[next_stop])
                vehicle.route[i] = (next_stop, vehicle.route[i][1], vehicle.route[i][2], arrival_time + s[next_stop])

            total_delivery -= feasible_delivery
            total_pickup -= feasible_pickup

        # Update remaining demand
        remaining_deliveries[customer] = total_delivery
        remaining_pickups[customer] = total_pickup

        if total_delivery > 0 or total_pickup > 0:
            uninsertable_customers.add(customer)

    # Handle Unused Vehicles Using Nearest Neighbor Heuristic
    unused_vehicles = [v for v in solution if len(v.route) == 1] # Only depot

    # Ensure both dicts have the same keys
    all_customers = set(remaining_deliveries.keys()) | set(remaining_pickups.keys())
    for cust in all_customers:
        remaining_deliveries.setdefault(cust, 0)
        remaining_pickups.setdefault(cust, 0)

    unserved_customers = [cust for cust in remaining_deliveries if remaining_deliveries[cust] > 0 or remaining_pickups[cust] > 0]

    if unused_vehicles and unserved_customers:
        print("Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...")

        # Sort unused vehicles by largest capacity
        unused_vehicles.sort(key=lambda v: v.capacity, reverse=True)

        for vehicle in unused_vehicles:
            if not unserved_customers:
                break

            vehicle.route = [(0, 0, 0, 0)] # Start from depot
            vehicle.current_time = 0

            while True:
                feasible_customers = [
                    c for c in unserved_customers
                    if vehicle.current_time + t[vehicle.route[-1][0]][c] <= b[c]
                    and (remaining_deliveries[c] > 0 or remaining_pickups[c] > 0)
                ]

                if not feasible_customers:
                    break

                next_customer = max(
                    feasible_customers,
                    key=lambda c: (-t[vehicle.route[-1][0]][c], remaining_deliveries[c] + remaining_pickups[c])
                )

                travel_time = t[vehicle.route[-1][0]][next_customer]
                arrival_time = max(vehicle.current_time + travel_time, a[next_customer])

                # Assign feasible delivery and pickup
                assigned_delivery = min(remaining_deliveries[next_customer], vehicle.full_vials)
                assigned_pickup = min(remaining_pickups[next_customer], vehicle.empty_space)

                # Skip if nothing to deliver/pick up
                if assigned_delivery == 0 and assigned_pickup == 0:
                    break

                # Update vehicle load
                vehicle.full_vials -= assigned_delivery
                vehicle.empty_vials += assigned_pickup
                vehicle.empty_space = vehicle.capacity - (vehicle.full_vials + vehicle.empty_vials)

                vehicle.route.append((next_customer, assigned_delivery, assigned_pickup, round(arrival_time, 2)))
                vehicle.current_time = round(arrival_time + s[next_customer], 2)

                # Update demand
                remaining_deliveries[next_customer] -= assigned_delivery
                remaining_pickups[next_customer] -= assigned_pickup

                if remaining_deliveries[next_customer] == 0 and remaining_pickups[next_customer] == 0:
                    unserved_customers.remove(next_customer)

                # Stop if vehicle is full or out of time
                if vehicle.empty_space == 0 or vehicle.current_time > max(b):
                    break

            # Return to depot if any customers were served
            if len(vehicle.route) > 1:
                return_time = t[vehicle.route[-1][0]][0]
                vehicle.route.append((0, 0, 0, round(vehicle.current_time + return_time, 2)))

    for vehicle in solution:
        if len(vehicle.route) > 1 and vehicle.route[0][0] != 0:
            vehicle.route.insert(0, (0, 0, 0, 0.0))

    return solution, remaining_deliveries, remaining_pickups, uninsertable_customers

# Example Usage: Reinsert Customers Removed by Random Removal
initial_deliveries = {i: d[i] for i in V_star}  # Convert delivery list to dictionary
initial_pickups = {i: p[i] for i in V_star}  # Convert pickup list to dictionary

modified_random, removed_random, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated), 0.5, deepcopy(initial_deliveries), deepcopy(initial_pickups)
)

updated_solution, final_deliveries, final_pickups, uninsertable_customers = regret_split_insertion(
    modified_random, removed_random, c, t, Q, initial_deliveries, initial_pickups, a, b, s
)

# Final Check
print("\n**Demand Satisfaction Check**")
unsatisfied_deliveries = {cust: amt for cust, amt in final_deliveries.items() if amt > 0}
unsatisfied_pickups = {cust: amt for cust, amt in final_pickups.items() if amt > 0}

if not unsatisfied_deliveries and not unsatisfied_pickups:
    print("All customer demands are satisfied!")
else:
    print("Some customer demands remain unsatisfied!")
    if unsatisfied_deliveries:
        print("Remaining Delivery Demand:", unsatisfied_deliveries)
    if unsatisfied_pickups:
        print("Remaining Pickup Demand:", unsatisfied_pickups)

# --- Print Results ---
print("\n**Updated Routes After Random Removal + Regret Split Insertion**")

for vehicle in updated_solution:
    print(f"Vehicle {vehicle.vehicle_id}: Route {[(c, d, p, float(t)) for (c, d, p, t) in vehicle.route]}")

Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...

**Demand Satisfaction Check**
All customer demands are satisfied!

**Updated Routes After Random Removal + Regret Split Insertion**
Vehicle 4: Route [(0, 0, 0, 0.0), (6, 10, 7, 62.0), (4, 34, 29, 78.0), (12, 8, 31, 407.0), (11, 33, 6, 438.0), (9, 0, 26, 458.0), (14, 0, 30, 407.0), (10, 0, 9, 407.0), (8, 0, 10, 442.0), (13, 10, 6, 457.0), (5, 33, 35, 495.0), (7, 0, 21, 517.0), (0, 0, 0, 547.0), (16, 36, 37, 566.0), (1, 21, 23, 592.0), (7, 16, 7, 625.0), (17, 27, 13, 647.0), (2, 11, 7, 664.0), (0, 0, 0, 696.0), (0, 0, 0, 704.0)]
Vehicle 2: Route [(0, 0, 0, 0.0), (9, 19, 0, 43.0), (15, 25, 21, 349.0), (8, 20, 0, 368.0), (12, 27, 0, 395.0), (3, 0, 22, 423.0), (0, 0, 0, 448.0), (0, 0, 0, 456.0)]
Vehicle 1: Route [(0, 0, 0, 0.0), (3, 32, 0, 92.0), (10, 28, 18, 349.0), (14, 37, 0, 378.0), (0, 0, 0, 398.0), (0, 0, 0, 406.0), (0, 0, 0, 414.0)]
Vehicle 7: Route [(0, 0, 0, 0.0), (16, 36, 29, 24.0), (16, 0, 8, 32

In [12]:
# Savings Insertion
import itertools

def check_time_window_feasibility(cluster1, cluster2, a, b, s, t):
    customers = list(set(cluster1 + cluster2))
    visited = set()
    current_time = 0
    current_node = 0
    route = []

    while len(visited) < len(customers):
        feasible_next = []

        for cust in customers:
            if cust in visited:
                continue
            travel_time = t[current_node][cust]
            arrival = current_time + travel_time
            if arrival <= b[cust]:
                adjusted_arrival = max(arrival, a[cust])
                heappush(feasible_next, (adjusted_arrival, cust, travel_time))

        if not feasible_next:
            return False  # no feasible customer to visit next

        # Choose the earliest feasible arrival
        next_arrival, next_customer, travel_time = heappop(feasible_next)
        route.append(next_customer)
        visited.add(next_customer)
        current_node = next_customer
        current_time = next_arrival + s[next_customer]

    return True

def savings_insertion(vehicles, removed_customers, c, t, Q, a, b, s, remaining_deliveries, remaining_pickups):
    import random

    gamma = 0.25
    clusters = []
    vehicle_cluster_map = {}

    # Create clusters from existing vehicle routes
    for vehicle in vehicles:
        cluster_customers = [stop[0] for stop in vehicle.route if stop[0] != 0]
        if cluster_customers:
            pickup_load = sum([stop[2] for stop in vehicle.route if stop[0] != 0])
            delivery_load = sum([stop[1] for stop in vehicle.route if stop[0] != 0])
            clusters.append({'customers': cluster_customers, 'pickup_load': pickup_load, 'delivery_load': delivery_load})
            for customer in cluster_customers:
                vehicle_cluster_map[customer] = clusters[-1]

    # Add singleton clusters for removed customers
    for customer in removed_customers:
        delivery = remaining_deliveries.get(customer, 0)
        pickup = remaining_pickups.get(customer, 0)
        cluster = {'customers': [customer], 'pickup_load': pickup, 'delivery_load': delivery}
        clusters.append(cluster)
        vehicle_cluster_map[customer] = cluster

    # Compute savings values
    savings_list = []
    customers = [cust for cluster in clusters for cust in cluster['customers']]
    for i in range(len(customers)):
        for j in range(i + 1, len(customers)):
            ci, cj = customers[i], customers[j]
            saving = c[0][ci] + c[0][cj] - c[ci][cj]
            savings_list.append((saving, ci, cj))

    # Sort savings values descending
    savings_list.sort(reverse=True)

    # Try merging clusters
    for saving, i, j in savings_list:
        cluster_i = vehicle_cluster_map[i]
        cluster_j = vehicle_cluster_map[j]

        if cluster_i == cluster_j:
            continue

        # Combined load
        total_delivery = cluster_i['delivery_load'] + cluster_j['delivery_load']
        total_pickup = cluster_i['pickup_load'] + cluster_j['pickup_load']

        # Full merge check
        if any(total_delivery + total_pickup <= v.capacity for v in vehicles):
            merged = cluster_i['customers'] + cluster_j['customers']
            if check_time_window_feasibility(cluster_i['customers'], cluster_j['customers'], a, b, s, t):
                # Merge
                new_cluster = {
                    'customers': merged,
                    'pickup_load': total_pickup,
                    'delivery_load': total_delivery
                }
                clusters.remove(cluster_i)
                clusters.remove(cluster_j)
                clusters.append(new_cluster)
                for customer in merged:
                    vehicle_cluster_map[customer] = new_cluster
            continue

        # Partial merge if one is singleton
        singleton, larger = None, None
        if len(cluster_i['customers']) == 1:
            singleton, larger = cluster_i, cluster_j
        elif len(cluster_j['customers']) == 1:
            singleton, larger = cluster_j, cluster_i

        if singleton and (total_delivery + total_pickup <= max(Q)):
            single_customer = singleton['customers'][0]
            partial_delivery = gamma * remaining_deliveries.get(single_customer, 0)
            partial_pickup = gamma * remaining_pickups.get(single_customer, 0)

            # Check time feasibility for partial merge
            test_cluster = larger['customers'] + [single_customer]
            if check_time_window_feasibility(larger['customers'], [single_customer], a, b, s, t):
                # Merge partially
                if single_customer not in larger['customers']:
                    larger['customers'].append(single_customer)
                larger['pickup_load'] += partial_pickup
                larger['delivery_load'] += partial_delivery

                remaining_deliveries[single_customer] -= partial_delivery
                remaining_pickups[single_customer] -= partial_pickup

                singleton['pickup_load'] -= partial_pickup
                singleton['delivery_load'] -= partial_delivery

                if remaining_deliveries[single_customer] <= 0 and remaining_pickups[single_customer] <= 0:
                    clusters.remove(singleton)

                vehicle_cluster_map[single_customer] = larger

    return clusters

def greedy_build_route(customers, c, t, a, b, s):
    unvisited = set(customers)
    route = []
    current_node = 0  # Start at depot
    current_time = 0
    total_cost = 0

    while unvisited:
        feasible_next = []
        for cust in unvisited:
            travel_time = t[current_node][cust]
            arrival_time = current_time + travel_time
            arrival_time = max(arrival_time, a[cust])
            if arrival_time <= b[cust]:
                cost = c[current_node][cust]
                feasible_next.append((cost, cust, arrival_time, travel_time))

        if not feasible_next:
            return None, float('inf')  # No feasible customer

        # Choose the nearest feasible customer by cost
        _, next_cust, arrival_time, travel_time = min(feasible_next)
        route.append((next_cust, arrival_time))
        total_cost += c[current_node][next_cust]
        current_time = arrival_time + s[next_cust]
        current_node = next_cust
        unvisited.remove(next_cust)

    total_cost += c[current_node][0]  # Return to depot
    return route, total_cost

def assign_vehicles_to_clusters(clusters, vehicles_updated, c, t, a, b, s):
    assigned_vehicle_ids = set()
    final_routes = []

    available_vehicles = deepcopy(vehicles_updated)

    for cluster in clusters:
        customers = cluster['customers']
        if not customers:
            continue

        route, cost = greedy_build_route(customers, c, t, a, b, s)
        if route is None:
            print(f"Could not build a feasible route for cluster: {customers}")
            continue

        # Shuffle vehicles to enhance randomness
        random.shuffle(available_vehicles)

        assigned = False
        for vehicle in available_vehicles:
            if vehicle.vehicle_id in assigned_vehicle_ids:
                continue
            if vehicle.capacity >= (cluster['pickup_load'] + cluster['delivery_load']):
                assigned_vehicle_ids.add(vehicle.vehicle_id)
                final_routes.append({
                    'vehicle_id': vehicle.vehicle_id,
                    'route': [(0, 0, 0, 0.0)] + [(cust, 0, 0, float(arr)) for cust, arr in route] + [(0, 0, 0, 0.0)],
                    'total_cost': round(cost, 2)
                })
                assigned = True
                break

        if not assigned:
            print(f"No available vehicle could serve cluster {customers} with load d={cluster['delivery_load']}, p={cluster['pickup_load']}")

    return final_routes

def fallback_vehicle_assignment(unassigned_clusters, available_vehicles, c, t, a, b, s, d, p):
    # Sort available vehicles by descending capacity
    sorted_vehicles = sorted(available_vehicles, key=lambda v: v.capacity, reverse=True)
    fallback_routes = []

    for cluster in unassigned_clusters:
        remaining_customers = set(cluster['customers'])

        for vehicle in sorted_vehicles:
            if not remaining_customers:
                break  # Cluster fully assigned

            # Try to serve as many customers as possible
            route = []
            current_node = 0
            current_time = 0
            current_delivery = 0
            current_pickup = 0
            assigned_customers = []
            unvisited = list(remaining_customers)

            while unvisited:
                feasible_next = []
                for cust in unvisited:
                    travel_time = t[current_node][cust]
                    arrival_time = current_time + travel_time
                    arrival_time = max(arrival_time, a[cust])

                    # Check time window and capacity feasibility
                    if (arrival_time <= b[cust] and
                        current_delivery + d[cust] <= vehicle.capacity and
                        current_pickup + p[cust] <= vehicle.capacity):
                        cost = c[current_node][cust]
                        feasible_next.append((cost, cust, arrival_time, travel_time))

                if not feasible_next:
                    break  # No more feasible insertions

                # Choose best feasible customer by cost
                _, next_cust, arrival_time, travel_time = min(feasible_next)
                route.append((next_cust, 0, 0, float(arrival_time)))
                current_node = next_cust
                current_time = arrival_time + s[next_cust]
                current_delivery += d[next_cust]
                current_pickup += p[next_cust]
                assigned_customers.append(next_cust)
                unvisited.remove(next_cust)

            if assigned_customers:
                route = [(0, 0, 0, 0.0)] + route + [(0, 0, 0, 0.0)]
                cost = sum(c[route[i][0]][route[i+1][0]] for i in range(len(route)-1))
                fallback_routes.append({
                    'vehicle_id': vehicle.vehicle_id,
                    'route': route,
                    'total_cost': round(cost, 2)
                })
                sorted_vehicles.remove(vehicle)
                remaining_customers -= set(assigned_customers)

    return fallback_routes

# Example
initial_deliveries = {i: d[i] for i in V_star}
initial_pickups = {i: p[i] for i in V_star}

# Apply random removal
modified_solution, removed_customers, updated_deliveries, updated_pickups = random_removal(
    deepcopy(vehicles_updated), 0.5, deepcopy(initial_deliveries), deepcopy(initial_pickups)
)

# Run savings insertion to form clusters
clusters = savings_insertion(
    modified_solution, removed_customers, c, t, Q, a, b, s,
    updated_deliveries, updated_pickups
)

final_routes = assign_vehicles_to_clusters(clusters, vehicles_updated, c, t, a, b, s)

# Identify used vehicle IDs
assigned_vehicle_ids = {r['vehicle_id'] for r in final_routes}
available_vehicles = [v for v in vehicles_updated if v.vehicle_id not in assigned_vehicle_ids]

# Identify unassigned clusters
assigned_customers = {cust for route in final_routes for cust, *_ in route['route'] if cust != 0}
unassigned_clusters = [cluster for cluster in clusters if not all(c in assigned_customers for c in cluster['customers'])]

# Fallback assignment using remaining vehicles
fallback_routes = fallback_vehicle_assignment(unassigned_clusters, available_vehicles, c, t, a, b, s, d, p)

# Combine all routes
final_routes.extend(fallback_routes)

print("Final Routes After Savings Insertion and Greedy Routing:")
for route_data in final_routes:
    print(f"Vehicle {route_data['vehicle_id']}: Route {route_data['route']} | Total Cost: {route_data['total_cost']}")

No available vehicle could serve cluster [4, 9, 13, 7, 16, 7, 17, 2] with load d=124, p=146
Final Routes After Savings Insertion and Greedy Routing:
Vehicle 4: Route [(0, 0, 0, 0.0), (14, 0, 0, 12.0), (9, 0, 0, 39.0), (15, 0, 0, 59.0), (13, 0, 0, 95.0), (0, 0, 0, 0.0)] | Total Cost: 9.67
Vehicle 2: Route [(0, 0, 0, 0.0), (14, 0, 0, 12.0), (11, 0, 0, 28.0), (10, 0, 0, 49.0), (12, 0, 0, 78.0), (1, 0, 0, 95.0), (5, 0, 0, 124.0), (6, 0, 0, 149.0), (8, 0, 0, 169.0), (15, 0, 0, 188.0), (3, 0, 0, 227.0), (0, 0, 0, 0.0)] | Total Cost: 16.93
Vehicle 1: Route [(0, 0, 0, 0.0), (16, 0, 0, 24.0), (7, 0, 0, 43.0), (17, 0, 0, 62.0), (2, 0, 0, 80.0), (9, 0, 0, 102.0), (13, 0, 0, 133.0), (0, 0, 0, 0.0)] | Total Cost: 10.88
Vehicle 6: Route [(0, 0, 0, 0.0), (4, 0, 0, 28.0), (0, 0, 0, 0.0)] | Total Cost: 5.21


In [31]:
# ALNS Procedure
def alns(
    initial_solution,
    removal_operators,
    insertion_operators,
    max_iterations,
    z,  # interval for updating weights
    n,  # stagnation threshold
    d, p, c, t, Q, a, b, s, vehicles_updated
):
    # Initialize weights and scores
    removal_weights = [1] * len(removal_operators)
    insertion_weights = [1] * len(insertion_operators)
    removal_scores = [0] * len(removal_operators)
    insertion_scores = [0] * len(insertion_operators)

    def roulette_wheel_selection(weights):
        total = sum(weights)
        r = random.uniform(0, total)
        cumulative_sum = 0
        for i, w in enumerate(weights):
            cumulative_sum += w
            if cumulative_sum >= r:
                return i
        return len(weights) - 1

    current_solution = deepcopy(initial_solution)
    best_solution = deepcopy(current_solution)
    stagnation_counter = 0

    for i in range(max_iterations):
        # Select and apply removal operator
        r_idx = roulette_wheel_selection(removal_weights)
        removal_op = removal_operators[r_idx]


        # Convert to dicts before passing to insertion
        updated_d = {i: d[i] for i in range(len(d))}
        updated_p = {i: p[i] for i in range(len(p))}

        if removal_op.__name__ == "related_removal":
            removed_sol, removed_customers, updated_d, updated_p = removal_op(
            deepcopy(current_solution), 0.5, distances, deepcopy(updated_d), deepcopy(updated_p)
        )
        elif removal_op.__name__ == "worst_removal":
            removed_sol, removed_customers, updated_d, updated_p = removal_op(
            deepcopy(current_solution), 0.5, distances, fixed_costs,
            deepcopy(updated_d), deepcopy(updated_p)
        )
        else:
            removed_sol, removed_customers, updated_d, updated_p = removal_op(
                deepcopy(current_solution), 0.5, deepcopy(updated_d), deepcopy(updated_p)
            )


        # Select and apply insertion operator
        ins_idx = roulette_wheel_selection(insertion_weights)
        insertion_op = insertion_operators[ins_idx]

        if insertion_op.__name__ == "savings_insertion":
            clusters = insertion_op(
                removed_sol, removed_customers, c, t, Q, a, b, s, updated_d, updated_p
            )
            new_solution = assign_vehicles_to_clusters(clusters, vehicles_updated, c, t, a, b, s)
        elif insertion_op.__name__ == "regret_split_insertion":
            new_solution, _, _, _ = insertion_op(
                removed_sol,
                removed_customers,
                c=c, t=t, vehicle_capacities=Q,
                remaining_deliveries=updated_d,
                remaining_pickups=updated_p,
                a=a, b=b, s=s
    )

        elif insertion_op.__name__ == "parallel_insertion":
            new_solution, _, _ = insertion_op(
                removed_sol,
                removed_customers,
                c=c, t=t, vehicle_capacities=Q,
                remaining_deliveries=updated_d,
                remaining_pickups=updated_p,
                a=a, b=b, s=s
            )

        # Evaluate improvement (objective: total cost)
        if isinstance(current_solution[0], dict):
            old_cost = sum(v['total_cost'] for v in current_solution)
        else:
            old_cost = sum(
            fixed_costs[v.vehicle_id] +
            sum(c[prev[0]][curr[0]] for prev, curr in zip(v.route[:-1], v.route[1:]))
            for v in current_solution if len(v.route) > 1
        )

        if isinstance(new_solution[0], dict):
            new_cost = sum(v['total_cost'] for v in new_solution)
        else:
            new_cost = sum(
            fixed_costs[v.vehicle_id] +
            sum(c[prev[0]][curr[0]] for prev, curr in zip(v.route[:-1], v.route[1:]))
            for v in new_solution if len(v.route) > 1
        )

        if new_cost < old_cost:
            current_solution = deepcopy(new_solution)
            stagnation_counter = 0
            removal_scores[r_idx] += 1
            insertion_scores[ins_idx] += 1
        else:
            stagnation_counter += 1

        if isinstance(best_solution[0], dict):
            best_cost = sum(v['total_cost'] for v in best_solution)
        else:
            best_cost = sum(
            fixed_costs[v.vehicle_id] +
            sum(c[prev[0]][curr[0]] for prev, curr in zip(v.route[:-1], v.route[1:]))
            for v in best_solution if len(v.route) > 1
        )

        if new_cost < best_cost:
            best_solution = deepcopy(new_solution)

        if i % z == 0 and i > 0:
            for j in range(len(removal_weights)):
                removal_weights[j] = max(1, removal_weights[j] * (1 + removal_scores[j]))
                removal_scores[j] = 0
            for j in range(len(insertion_weights)):
                insertion_weights[j] = max(1, insertion_weights[j] * (1 + insertion_scores[j]))
                insertion_scores[j] = 0

        if stagnation_counter >= n:
            current_solution = deepcopy(best_solution)
            stagnation_counter = 0

    return best_solution

In [32]:
# print call to function
best_solution = alns(
    initial_solution=deepcopy(vehicles_updated),
    removal_operators=[random_removal, related_removal, worst_removal],
    insertion_operators = [parallel_insertion, regret_split_insertion, savings_insertion],
    max_iterations=200,
    z=20,
    n=30,
    d=d, p=p, c=c, t=t, Q=Q, a=a, b=b, s=s, vehicles_updated=vehicles_updated
)

print("✅ Final Best Solution:")
for route_data in best_solution:
    print(f"Vehicle {route_data['vehicle_id']}: Route {route_data['route']} | Total Cost: {route_data['total_cost']}")

Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
Assigning remaining customers to unused vehicles using Nearest Neighbor Heuristic...
No available vehicle could serve cluster [5, 6, 9, 8, 16, 17] with load d=106, p=128


AttributeError: 'dict' object has no attribute 'route'